Basic Project Outline:

1) Introduction - DONE
2) Import map data - DONE
3) Create sample of of map data - DONE
4) Basic overview of data
5) List/explain 2 or 3 problems found in data (eg. abbreviated street names, incorrect postal codes, ???)
6) Correct problem 1
7) Correct problem 2
8) Correct problem 3 (if applicable)
9) Export data as csv file
10) Import csv file into sql
11) Compute overview of staistics of file:
    a) size of the file
    b) number of unique users
    c) number of nodes and ways
    d) number of chosen type of nodes, like cafes, shops etc.
    e) Top 10 contributing users
    f) Most popular cuisines?
12) Conclusion
    a) Ideas for improving the data set more
    b) discussion about the benefits as well as some anticipated problems in implementing the improvement

<h1>Introduction</h1>

In this project I will be exploring OpenStreetMap data for Irvine, California. I chose this map data because Irvine is my current city of residence. I am already pretty familiar with the area, which will potentially help me recognize some of the bad data in the set. At the same time, I'm curious about some aspects of my city and I will use this data set to glean some new insights about it.

The data consists of XML elements called "nodes" (points of interest) and "ways" (linear features and area boundaries). Each element can have one or more tags associated with it. These tags provide more details and information about the element.  Additional information about the data structure can be found here: https://wiki.openstreetmap.org/wiki/OSM_XML

I will first explore the raw XML data using Python and audit tags that are appear to have errors, and tags that I'm interested in analyzing in more detail. Due to the scope of this project, I will not be cleaning every error I find, but will focus on two or three to meet the requirements of the project. After identifying and cleaning the problem areas, I will export the data to CSV files, then import it into a SQLite database. Finally, I will query the database to gain greater insights.



<h1>Map Area</h1>

On OpenStreetMap.org, the area I selected to include as much of Irvine I could without too much data from other cities is bounded within the following coordinates:

Longitude: -117.8792, -117.7281
Latitude: 33.7301, 33.6135

The map can be found here: https://www.openstreetmap.org/search?query=Irvine%2C%20CA#map=12/33.7299/-117.8232. The underlying data was downloaded using the Overpass API, at this link: https://overpass-api.de/api/map?bbox=-117.8792,33.6135,-117.7281,33.7301.



In [1]:
import os
os.getcwd()

'C:\\Users\\randy'

In [2]:
os.chdir("D:\Documents\School - WGU\Term 4\C750\Project")
os.getcwd()

'D:\\Documents\\School - WGU\\Term 4\\C750\\Project'

<h1>Data Exploration and Cleaning</h1>

Using the 'OSM_sampling.py' script provided by Udacity, I will create a sample file of every 10th element from the original file map data.

In [3]:
#Importing modules
import xml.etree.ElementTree as ET
from collections import defaultdict
import re
import pprint
import re
import xml.etree.cElementTree as ET
import csv
import codecs
import cerberus
import schema

In [4]:
#Import osm file and create sample dataset

#!/usr/bin/env python
# -*- coding: utf-8 -*-

OSM_FILE = "map.osm"
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

            
with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write(b'<osm>\n  ')
    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))
    output.write(b'</osm>')

Next, we will parse the sample dataset to count the unique element types. 

In [5]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

def test():

    tags = count_tags('sample.osm')
    pprint.pprint(tags)
    

if __name__ == "__main__":
    test()

{'member': 2608,
 'nd': 38943,
 'node': 33409,
 'osm': 1,
 'relation': 145,
 'tag': 35851,
 'way': 4695}


By using the count_tags function, I got the number of unique tags shown. The ones that I expect to be of the most interest to me are:

<ul>
    <li>Nodes</li>
    <li>Tags</li>
    <li>Ways</li>
    <li>Members</li>
</ul>


To begin, I will check the "k" value for each "<tag>" to see if there are any potential problems. There are three regular expressions: 
    
<ol>
    <li>"lower", for tags that contain only lowercase letters and are valid,</li>
    <li>"lower_colon", for otherwise valid tags with a colon in their names,</li>
    <li>"problemchars", for tags with problematic characters, and</li>
    <li>"other", for other tags that do not fall into the other three categories.</li>
</ol>
    
We need a count of each of these four tag categories in a dictionary.

In [6]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] +=1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon']+=1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars']+=1
        else:
            keys['other']+=1
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



def test():
    keys = process_map('sample.osm')
    pprint.pprint(keys)

if __name__ == "__main__":
    test()

{'lower': 17335, 'lower_colon': 18152, 'other': 364, 'problemchars': 0}


As we continue to explore the data, let's find out how many unique users have contributed to the data in this area.

In [7]:
def get_user(element):
    return


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            continue

    return users


def test():

    users = process_map('sample.osm')
    pprint.pprint(len(users))

if __name__ == "__main__":
    test()

769


Now, we need to check the validity and consistency of the street names.

In [8]:
OSMFILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Trail", "Parkway", "Commons", "Layout", "Main", "Broadway", "Plaza", "Park"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def print_sorted_dic(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print("%s: %d" % (k, v))

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit():
#    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(OSMFILE, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))
#    osm_file.close()
#    return street_types

if __name__ == '__main__':
    audit()

{'Aberdeen': {'Aberdeen'},
 'Abeto': {'Abeto'},
 'Ada': {'Ada'},
 'Adamo': {'Adamo'},
 'Aguilar': {'Aguilar'},
 'Aims': {'Aims'},
 'Alaska': {'Alaska'},
 'Alder': {'Speckled Alder', 'Alder'},
 'Alegria': {'Alegria'},
 'Alevera': {'Alevera'},
 'Alhambra': {'Alhambra'},
 'Allegheny': {'Allegheny'},
 'Allium': {'Allium'},
 'Alondra': {'Alondra'},
 'Amargosa': {'Amargosa'},
 'Ambiance': {'Ambiance'},
 'Ambroise': {'Ambroise'},
 'Amistad': {'Amistad'},
 'Ancona': {'Ancona'},
 'Andalucia': {'Andalucia'},
 'Andante': {'Del Andante'},
 'Andreta': {'Caminito Andreta'},
 'Antietam': {'Antietam'},
 'Antonio': {'San Antonio'},
 'Anzio': {'Anzio'},
 'Apache': {'Apache'},
 'Appomattox': {'Appomattox'},
 'Arbor': {'Shady Arbor', 'Grape Arbor'},
 'Arboretum': {'Arboretum'},
 'Arborglen': {'Arborglen'},
 'Arborside': {'Arborside'},
 'Arches': {'Arches'},
 'Aristotle': {'Aristotle'},
 'Arizona': {'Arizona'},
 'Arlington': {'Arlington'},
 'Ash': {'Maple Ash'},
 'Ashdale': {'Ashdale'},
 'Ashford': {'Ashfo

The street name data is remarkably clean, though not perfect. Let's clean it up.

In [9]:
mapping = {"St": "Street",
           "ST": "Street",
           "St.": "Street",
           "St,": "Street",
           "Street.": "Street",
           "street": "Street",
           "Sq": "Square",
           "Rd.": "Road",
           "Rd": "Road",
           "Ave": "Avenue",
           "DR.": "Drive"
           }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):

        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    for key, value in mapping.items():
        if re.search(key, name):
            name = re.sub(street_type_re, value, name)

    return name


def test():
    st_types = audit(OSMFILE)
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)


if __name__ == '__main__':
    test()

Carrington Way => Carrington Way
Sandburg Way => Sandburg Way
Wheaton Way => Wheaton Way
Camden Way => Camden Way
Mayfield Way => Mayfield Way
Fairfax Way => Fairfax Way
Scientific Way => Scientific Way
Byron Way => Byron Way
Taylor Way => Taylor Way
Wesley Way => Wesley Way
6685 Marina Way => 6685 Marina Way
Lafayette Way => Lafayette Way
Manchester Way => Manchester Way
Liberty => Liberty
Honors => Honors
Romano => Romano
Exchange => Exchange
Commerce => Commerce
Quail Ridge => Quail Ridge
Visions => Visions
Roosevelt => Roosevelt
Theory => Theory
Amistad => Amistad
Peters Canyon => Peters Canyon
Silk Tree => Silk Tree
Knollcrest => Knollcrest
Rockefeller => Rockefeller
Grandoaks Circle => Grandoaks Circle
Danberry Circle => Danberry Circle
Gateway Circle => Gateway Circle
Kimberly Circle => Kimberly Circle
Dartmouth Circle => Dartmouth Circle
Enclave Circle => Enclave Circle
Munton Circle => Munton Circle
Discovery => Discovery
Bunsen => Bunsen
Parkway Loop => Parkway Loop
Vintage =

This is better, though it does introduce a new problem: names that start with "St" (e.g. "Stanza") are changed to "Street," which is obviously not what was intended. If this project's purpose was to fully clean the data 100%, I would add code to exclude those those exceptions. However, as a proof of concept for the methodology this was successful, so we will move on. 

Let's look at the zip codes in the dataset.

In [10]:
# Here we create a dictionary of our postal codes
def audit_postal_code(postal_code_types, postal_code):  
    if not postal_code.isupper() or ' ' not in postal_code:
        postal_code_types['Postal Codes'].add(postal_code)
    else:
        postal_code_types['other'].add(postal_code)
    return postal_code_types

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit(filename):
    f = (filename)
    postal_code_types = defaultdict(set)
    
    for event, element in ET.iterparse(f, events=("start",)):
        if element.tag =="way":
            for tag in element.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(postal_code_types, tag.attrib['v'])
    print(dict(postal_code_types))

if __name__ == '__main__':
    audit(OSMFILE)

{'Postal Codes': {'92603', '92705', '92653', '92604', '92612', '92637', '92620', '92782', '92606', '92618', '92617', '92614', '92663', '92780', '92714', '92707', '92697', '92660', '92626', '92602'}}


In [11]:
OSMFILE = "sample.osm"
postal_code_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["92602", "92603", "92604", "92606", "92612", "92614", "92616", "92617", 
            "92618", "92619", "92620", "92623", "92650", "92697"]


def audit_postal_code(postal_code_types, postal_codes):
    m = postal_code_type_re.search(postal_codes)
    if m:
        postal_code = m.group()
        if postal_code not in expected:
            postal_code_types[postal_code].add(postal_code)

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit():
    postal_codes = defaultdict(set)
    for event, elem in ET.iterparse(OSMFILE, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(postal_codes, tag.attrib['v'])
    
    print("Postal codes in data set that are not in Irvine")
    pprint.pprint(dict(postal_codes))

if __name__ == '__main__':
    audit()

Postal codes in data set that are not in Irvine
{'92520': {'92520'},
 '92625': {'92625'},
 '92626': {'92626'},
 '92630': {'92630'},
 '92637': {'92637'},
 '92653': {'92653'},
 '92657': {'92657'},
 '92660': {'92660'},
 '92663': {'92663'},
 '92705': {'92705'},
 '92707': {'92707'},
 '92714': {'92714'},
 '92780': {'92780'},
 '92782': {'92782'}}


Let's see what unique keys there are, and how many.

In [12]:
def unique_keys(filename):
    distinct_keys=[]
    count=1

    EL=get_element(filename, tags=('node', 'way', 'relation'))
    for element in EL:
        if element.tag=='node' or element.tag=='way':
            for tag in element.iter('tag'):
                if tag.attrib['k'] not in distinct_keys:
                    distinct_keys.append(tag.attrib['k'])
                    count+=1
    distinct_keys.sort()
    print("Total number of unique keys (tag attrib['k'])is {}:".format(count))
    
#    return distinct_keys
      
    pprint.pprint(distinct_keys)
    
                
unique_keys(OSMFILE)  # Using Sample file as input to audit the addr:street key

Total number of unique keys (tag attrib['k'])is 324:
['FIXME',
 'FIXME:NHS',
 'FIXME:bicycle',
 'FMMP_modified',
 'FMMP_reviewed',
 'NHS',
 'abandoned:highway',
 'access',
 'access:disabled',
 'access:lanes',
 'acres',
 'addr:city',
 'addr:country',
 'addr:county',
 'addr:full',
 'addr:housenumber',
 'addr:postcode',
 'addr:state',
 'addr:street',
 'addr:unit',
 'admin_level',
 'aeroway',
 'alt_name',
 'amenity',
 'area',
 'area:highway',
 'attribution',
 'backrest',
 'bad_maxspeed',
 'barrier',
 'basin',
 'bench',
 'bicycle',
 'bicycle_parking',
 'bin',
 'boundary',
 'brand',
 'brand:wikidata',
 'brand:wikipedia',
 'bridge',
 'bridge_name',
 'bridge_ref',
 'building',
 'building:flats',
 'building:levels',
 'building:part',
 'bus',
 'button_operated',
 'cables',
 'caltrans:district',
 'caltrans:dynsegpm',
 'caltrans:type',
 'capacity',
 'capacity:disabled',
 'centre_turn_lane',
 'change:lanes',
 'check_date',
 'circuits',
 'construction',
 'conveying',
 'courts',
 'covered',
 'craft',

A few "Fix Me" tags. I won't be cleaning those today, but they definitely indicate a problem with the data set that it would help users of OpenStreetMaps to resolve.

Let's also take a look at phone number formats before we do the analysis and cleaning of the full data set.


In [13]:
def is_phone(elem):
    return (elem.attrib['k'] == "phone" or elem.attrib['k'] == "contact:phone")

def audit_phone_number_formats(phone_number_formats, phone_number):

    # Convert any digit to an 'X' sign (e.g. '(212) 333-3100' becomes '(XXX) XXX-XXXX')
    phone_number_format = re.sub('\d', 'X', phone_number)
    phone_number_formats[phone_number_format] += 1

def update_phone_number(phone_number):
    # Keep the first phone number if more than one is present
    if re.search(r';', phone_number):
        phone_number = phone_number.split(';')[0] # Phone numbers are separated by ';'
    elif re.search(r'/', phone_number):
        phone_number = phone_number.split('/')[0] # Phone numbers are separated by '/'
    
    digits = re.sub('\D', '', phone_number)
    if len(digits) == 11: # 1XXXXXXXXXX
        return '+' + digits[0] + '-' + digits[1:4] + '-' + digits[4:7] + '-' + digits[7:]
    elif len(digits) == 10: # XXXXXXXXXX
        return '+1' + '-' + digits[:3] + '-' + digits[3:6] + '-' + digits[6:]
    elif len(digits) == 12: # 01XXXXXXXXXX
        return '+' + digits[1] + '-' + digits[2:5] + '-' + digits[5:8] + '-' + digits[8:]
    elif len(digits) == 13: # 001XXXXXXXXXX
        return '+' + digits[2] + '-' + digits[3:6] + '-' + digits[6:9] + '-' + digits[9:]
    else:
        return phone_number
    
def audit(osmfile):
    osm_file = OSMFILE
    phone_number_formats = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=('start',)):

        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                # Audit phone numbers
                if is_phone(tag):
                    audit_phone_number_formats(phone_number_formats, tag.attrib['v'])
    pprint.pprint(dict(phone_number_formats))

audit(OSM_FILE)

{'(XXX) XXX-XXXX': 3,
 '+X (XXX) XXX-XXXX': 2,
 '+X XXX XXX XXXX': 3,
 '+X-XXX-XXX-XXXX': 3,
 'XXX-XXX-XXXX': 2}


<h1>Data Reshaping and Exporting to CSV Files</h1>

OK, it's time to actually reshape/clean the data and get it exported to CSV files.

In [22]:
# Schema

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


In [25]:
OSM_PATH = "sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    
    if element.tag=='node':
        for key in  element.attrib.keys():
            if key in node_attr_fields:
                node_attribs[key] = element.attrib[key]
#         print node_attribs

    if  element.findall('tag'):
        for elem in element.iter('tag'):
            tag={}
            tag['id']=element.attrib['id']
            tag['value']=elem.attrib['v']
            if ':' in elem.attrib['k']:
                type = re.split(':',elem.attrib['k'])[0]
                tag['key']=elem.attrib['k'][len(type)+1:]
                tag['type']=type
            else:
                tag['key']=elem.attrib['k']
                tag['type']='regular'
            if  LOWER_COLON.match(tag['key']) and PROBLEMCHARS.match(tag['key']):
                print(tag['key'])
            else:
                tags.append(tag)
#     print tags
#     print '\n'
    
    if element.tag=='way':
        for key in element.attrib.keys():
            if key in way_attr_fields:
                way_attribs[key] = element.attrib[key]
#         print way_attribs

   
    
    if element.tag =='way':
       
        i=0  
        for tag in element.iter('nd'):
            temp={}
            temp['id']=element.attrib['id']
            temp['node_id']= tag.attrib['ref']
            temp['position']= i
            i+=1
            way_nodes.append(temp)
        
        
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

        
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=schema):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)


NameError: name 'unicode' is not defined